In [ ]:
import os
import cv2
import torch
import json
import random
import shutil
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import sklearn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split
import logging
logging.disable(logging.DEBUG)

# ERROR
#### （1）标签溢出问题：即识别出来的坐标轴标签数量大于实际坐标轴标签数量，此问题易于解决，且训练集因此准确度下降6.7%：
* a、将坐标轴识别为图中的点，即圈圈圆圆圈圈（原因是圈太多了，圈数大于坐标轴标签数）
* b、将坐标轴识别为标题（原因是标题被识别成了好几段，段数大于坐标轴标签数）
* c、将纵坐标最下方的点或纵轴标题的一小段识别入横坐标轴标签中，最为常见（原因是最下方的点的纵坐标和横坐标轴标签的纵坐标接近）
* d、将无字符的地方识别为奇怪的字符，置信度低，但会混入横坐标轴标签中
* e、坐标轴识别的容错度d太大，导致较近的别的字符串也被识别进去

#### （2）横纵坐标标签多行问题
#### （3）横坐标相近的黏在一起，或许可以根据比例切开
#### （4）分类应该切96而不是64（fixed）
#### （5）特殊符号问题
* a、优化被杀掉字符如引号的输出，使其被允许
* b、将明显错误的特殊符号进行更合理的替换

#### （6）Exception处理：或许有许多exception只是单纯是意外情况，发现并处理可能能增加准确性
#### （7）OCR的训练及参数调整：
* a、detection的训练，暂定输入全部训练集，可调参数
* b、recognition的训练，暂定0-5000而已，可调参数
* c、调用ocr时有许多参数可以研究

#### （8）图片糊度问题，或许可以做些增强再ocr？
#### （9）多角度OCR，可以多OCR一些旋转角度，增加识别出的可能
#### （10）集成OCR模型，可以增添新的OCR同时输出结果取较好
#### （11）特殊图表问题
* a、直方图的标签处理问题
* b、折线图横轴标签上方无点问题
* c、其他的特殊情况（待补充）

#### （12）各种的find points优化
#### （13）各种的点插值优化
#### （14）纵轴还未尝试45度旋转，效果未必更佳
#### （15）错误输出猜答案的时候的刻意填充，猜分数更高的
#### （16）横轴漏标签，或许可以依照距离进行补齐
#### （17）横轴标签数量太少，会识别为别的东西
#### （18）可以考虑多截取几个框分类进行投票，可能分类更准

# install OCR Packages

In [ ]:
!pip install paddlepaddle-gpu --no-index --find-links=file:/kaggle/input/package37/package
!pip install /kaggle/input/packageocr/packageOCR/fire-0.5.0/fire-0.5.0
!pip install /kaggle/input/packageocr/packageOCR/ffmpy-0.3.0/ffmpy-0.3.0
!pip install /kaggle/input/packageocr/packageOCR/python-docx-0.8.11/python-docx-0.8.11
!pip install /kaggle/input/packageocr/packageOCR/fonttools-4.38.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/PyMuPDF-1.20.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/pdf2docx-0.5.6-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/Babel-2.12.1-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/MarkupSafe-2.1.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/Jinja2-3.1.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/pytz-2023.3-py2.py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/Werkzeug-2.2.3-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/itsdangerous-2.1.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/click-8.1.3-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/importlib_metadata-6.6.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/Flask-2.2.5-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/flask_babel-3.1.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/x2paddle-1.4.1-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/rarfile-4.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/bce_python_sdk-0.8.83-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/protobuf-3.20.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/rapidfuzz-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/h11-0.12.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/uvicorn-0.22.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/sniffio-1.3.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/anyio-3.6.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/httpcore-0.15.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/httpx-0.24.1-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/python_multipart-0.0.6-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/websockets-11.0.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/mdurl-0.1.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/markdown_it_py-2.2.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/typing_extensions-4.6.1-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/pydantic-1.10.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/orjson-3.8.13-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/starlette-0.27.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/fastapi-0.95.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/mdit_py_plugins-0.3.5-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/uc_micro_py-1.0.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/linkify_it_py-2.0.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/gradio-3.11.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/python_rapidjson-1.10-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/tritonclient-2.33.0-py3-none-manylinux1_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/paddle2onnx-1.0.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/charset_normalizer-3.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/async_timeout-4.0.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/asynctest-0.13.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/frozenlist-1.3.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/aiosignal-1.3.1-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/aiohttp-3.8.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/grpcio-1.54.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/Brotli-1.0.9-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/geventhttpclient-2.0.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/visualdl-2.5.2-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/opencv_contrib_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/attrdict-2.0.1-py2.py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/lmdb-1.4.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/cssutils-2.6.0-py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/cssselect-1.2.0-py2.py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/premailer-3.10.0-py2.py3-none-any.whl
!pip install /kaggle/input/packageocr/packageOCR/pyclipper-1.3.0.post4-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
!pip install /kaggle/input/packageocr/packageOCR/paddleocr-2.6.1.3-py3-none-any.whl

# Transform Type Label

In [ ]:
def label_transform(label):
    if label=='dot':
        return 0
    elif label=='horizontal_bar':
        return 1
    elif label=='vertical_bar':
        return 2
    elif label=='line':
        return 3
    elif label=='scatter':
        return 4
    elif label==0:
        return 'dot'
    elif label==1:
        return 'horizontal_bar'
    elif label==2:
        return 'vertical_bar'
    elif label==3:
        return 'line'
    elif label==4:
        return 'scatter'
    else:
        raise ValueError("Input value is not a label!")

# data path

In [ ]:
data_path = '/kaggle/input/benetech-making-graphs-accessible/' 

# Read TrainRawdata

In [ ]:
class RawData:

    __all_train_filenames=None
    __train_imagenames=None
    __test_imagenames=None
    __train_data=None
    __begin_index=0
    __end_index=None
    __train_labels=None
    __train_source=None

    @staticmethod
    def get_all_train_filenames():
        if RawData.__all_train_filenames is None:
            RawData.__all_train_filenames= os.listdir(data_path+"train"+"/annotations")
        return RawData.__all_train_filenames

    @staticmethod
    def get_train_data(begin=0,end=None):
        if RawData.__train_data is None or begin!=RawData.__begin_index or end!=RawData.__end_index:
            RawData.__begin_index=begin
            RawData.__end_index=end
            RawData.__train_data=[]
            RawData.get_all_train_filenames()
            for dataname in RawData.__all_train_filenames[begin:end]:
                f = open(data_path+"train/annotations/"+dataname, 'r')
                content = f.read()
                a = json.loads(content)
                f.close()
                a['image-name']=dataname[0:-5]+".jpg"
                RawData.__train_data.append(a)
        return RawData.__train_data
    
    @staticmethod
    def get_train_imagenames():
        if RawData.__train_data is None:
            raise NameError("Without definition of RawData.__train_data,use get_train_data() to define it!")
        RawData.__train_imagenames=[]
        for i in RawData.__train_data:
            RawData.__train_imagenames.append(i['image-name'])
        return RawData.__train_imagenames

    @staticmethod
    def get_train_labels():
        if RawData.__train_data is None:
            raise NameError("Without definition of RawData.__train_data,use get_train_data() to define it!")
        RawData.__train_labels=[]
        for i in RawData.__train_data:
            RawData.__train_labels.append(label_transform(i['chart-type']))
        
        return RawData.__train_labels


# Rotate

In [ ]:
def r45(img):
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), -45, 1)
    l = int((rows + cols) * 0.707106)
    M[0, 2] += (l - cols) * 0.5
    M[1, 2] += (l - rows) * 0.5
    return cv2.warpAffine(img, M, (l, l)), M[:, 2]

def recover(x1, y1, m):
    x=int(x1)
    y=int(y1)
    x -= int(m[0])
    y -= int(m[1])
    x, y = 0.707106 * (x+y), 0.707106 * (y-x)
    if x<0:x=0
    if y<0:y=0
    return int(x), int(y)

# OCR Initialization

In [ ]:
import paddle
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en",drop_score=0,cls_model_dir='/kaggle/input/pretrain-old/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer',det_model_dir='/kaggle/input/pretrain-old/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer',rec_model_dir='/kaggle/input/pretrain-old/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer')

# Load Test Pictures

In [ ]:
class TestData:

    __test_imagenames=None
    __test_data=None
    __begin_index=0
    __end_index=None
    
    @staticmethod
    def get_test_imagenames():
        if TestData.__test_imagenames is None:
            TestData.__test_imagenames= os.listdir(data_path+"test"+"/images")
        return TestData.__test_imagenames
    
    @staticmethod
    def get_test_data(begin=0,end=None):
        if TestData.__test_data is None or begin!=TestData.__begin_index or end!=TestData.__end_index:
            TestData.__begin_index=begin
            TestData.__end_index=end
            TestData.__test_data=[]
            TestData.get_test_imagenames()
            for dataname in TestData.__test_imagenames[begin:end]:
                a={}
                a['image-name']=dataname
                a['image']= cv2.imread(data_path+'test/images/'+dataname)
                TestData.__test_data.append(a)
        return TestData.__test_data
    
    def add_ocr_results():
        if TestData.__test_data is None:
            raise NameError("Without definition of TestData.__test_data,use get_test_data() to define it!")
        for i in range(0,len(TestData.__test_data)):
            TestData.__test_data[i]['ocr']=ocr.ocr(TestData.__test_data[i]['image'], cls=True)
            image45,TestData.__test_data[i]['m']=r45(TestData.__test_data[i]['image'])
            TestData.__test_data[i]['ocr45']=ocr.ocr(image45, cls=True)
            for j in range(0,len(TestData.__test_data[i]['ocr45'][0])):
                TestData.__test_data[i]['ocr45'][0][j][0][0][0],TestData.__test_data[i]['ocr45'][0][j][0][0][1]=recover(TestData.__test_data[i]['ocr45'][0][j][0][0][0],TestData.__test_data[i]['ocr45'][0][j][0][0][1],TestData.__test_data[i]['m'])
                TestData.__test_data[i]['ocr45'][0][j][0][1][0],TestData.__test_data[i]['ocr45'][0][j][0][1][1]=recover(TestData.__test_data[i]['ocr45'][0][j][0][1][0],TestData.__test_data[i]['ocr45'][0][j][0][1][1],TestData.__test_data[i]['m'])
                TestData.__test_data[i]['ocr45'][0][j][0][2][0],TestData.__test_data[i]['ocr45'][0][j][0][2][1]=recover(TestData.__test_data[i]['ocr45'][0][j][0][2][0],TestData.__test_data[i]['ocr45'][0][j][0][2][1],TestData.__test_data[i]['m'])
                TestData.__test_data[i]['ocr45'][0][j][0][3][0],TestData.__test_data[i]['ocr45'][0][j][0][3][1]=recover(TestData.__test_data[i]['ocr45'][0][j][0][3][0],TestData.__test_data[i]['ocr45'][0][j][0][3][1],TestData.__test_data[i]['m'])
            
    def row_detect(d=5,d45=10,ord=1):
        for i in range(0,len(TestData.__test_data)):
            list_ca=[]
            list_ca_st=[]
            for item in TestData.__test_data[i]['ocr'][0]:
                row_item=np.array([item[0][0][1],item[0][1][1],item[0][2][1],item[0][3][1]])
                min1=np.min(row_item)
                min2 =np.sort(row_item)[1]
                row_item=np.array([min1,min2])
                state=0
                for ca in range(0,len(list_ca_st)):
                    if np.linalg.norm(row_item-list_ca_st[ca], ord=ord)<=d:
                        list_ca[ca].append(item)
                        state=1
                if state==0:
                    list_ca.append([item])
                    list_ca_st.append(row_item)
            axis_num=0;
            axis_t=0;
            for axis in list_ca:
                if len(axis)>axis_num:
                    axis_t=axis
                    axis_num=len(axis)        
            list_ca=[]
            list_ca_st=[]
            for item in TestData.__test_data[i]['ocr45'][0]:
                row_item=np.array([item[0][0][1],item[0][1][1],item[0][2][1],item[0][3][1]])
                min1=np.min(row_item)
                min2 =np.sort(row_item)[1]
                row_item=np.array([min1,min2])
                state=0
                for ca in range(0,len(list_ca_st)):
                    if np.linalg.norm(row_item-list_ca_st[ca], ord=ord)<=d45:
                        list_ca[ca].append(item)
                        state=1
                if state==0:
                    list_ca.append([item])
                    list_ca_st.append(row_item)
            axis_num2=0;
            axis_t2=0;
            for axis in list_ca:
                if len(axis)>axis_num2:
                    axis_t2=axis
                    axis_num2=len(axis)
            
            if axis_num2>axis_num:
                TestData.__test_data[i]['row_axis_num']=axis_num2
                TestData.__test_data[i]['row_axis']=axis_t2
            else:
                TestData.__test_data[i]['row_axis_num']=axis_num
                TestData.__test_data[i]['row_axis']=axis_t
    
    def col_detect(d=5,ord=1):
        for i in range(0,len(TestData.__test_data)):
            list_ca=[]
            list_ca_st=[]
            for item in TestData.__test_data[i]['ocr'][0]:
                col_item=np.array([item[0][0][0],item[0][1][0],item[0][2][0],item[0][3][0]])
                max1=np.max(col_item)
                max2 =np.sort(col_item)[-2]
                col_item=np.array([max1,max2])
                state=0
                for ca in range(0,len(list_ca_st)):
                    if np.linalg.norm(col_item-list_ca_st[ca], ord=ord)<=d:
                        list_ca[ca].append(item)
                        state=1
                if state==0:
                    list_ca.append([item])
                    list_ca_st.append(col_item)
            axis_num=0;
            axis_t=0;
            for axis in list_ca:
                if len(axis)>axis_num:
                    axis_t=axis
                    axis_num=len(axis)
            TestData.__test_data[i]['col_axis_num']=axis_num
            TestData.__test_data[i]['col_axis']=axis_t


# Data Initialization

In [ ]:
data=TestData.get_test_data()

# OCR

In [ ]:
TestData.add_ocr_results()
TestData.col_detect(30,1)
TestData.row_detect(20,20,1)
data=TestData.get_test_data()

# Train and Test Classification Model

In [ ]:
class Data(Dataset):

    def __init__(self,type_,transform,begin=0,end=None):

        self.type = type_
        self.train_data=RawData.get_train_data(begin,end)
        self.train_labels= RawData.get_train_labels()
        self.train_imagenames = RawData.get_train_imagenames()
        self.transform = transform

    def __getitem__(self, index):

        if self.type == "train":
            label = self.train_labels[index]
            image = cv2.imread(data_path+"train/"+"images/"+self.train_imagenames[index])

        elif self.type == "test":
            label = 0
            image = data[index]['image']
            if image.shape[0]<96 or image.shape[1]<96:
                image=cv2.resize(image, (100,100))

        return label, self.transform(image)

    def __len__(self):
        len_ = 0
        if self.type == "train":
            len_ = len(self.train_imagenames)
        elif self.type == "test":
            len_ = len(data)

        return len_

class residual_block(nn.Module):
    """残差网络
    """
    def __init__(self, channel, dropout=False):
        super(residual_block, self).__init__()

        self.cov1 = nn.Conv2d(channel, channel, 3, 1, 1)
        self.cov2 = nn.Conv2d(channel, channel, 3, 1, 1)

        self.nor1 = nn.BatchNorm2d(channel)
        self.nor2 = nn.BatchNorm2d(channel)
        self.relu = nn.ReLU()
        self.dropout = dropout
        if dropout:
            self.drop = nn.Dropout(p=0.2)

    def forward(self, x):
        x_ = self.cov1(x)
        x_ = self.nor1(x_)
        x_ = self.relu(x_)

        x_ = self.cov2(x_)
        x_ = self.nor2(x_)
        x_ = self.relu(x_)

        if self.dropout:
            x_ = self.drop(x_)

        x = x + x_
        return x
    

class nor_cov(nn.Module):
    """单层卷积网络
    torch.nn.Module是pytorch一个封装好的类,其中用户通过重载__init__和forward完成自定义的网络结构
    参考连接:https://blog.csdn.net/qq_27825451/article/details/90550890
    网络中具有可学习参数的层(如全连接层、卷积层等)放在构造函数__init__()中
    nn.Module里还有一些如parameters、add_module之类的函数可以使用
    """
    def __init__(self, in_channel, out_channel, dropout=False, normalize=False):
        #调用父类的init，解决多重继承问题，格式是super(类名,self).__init__()，也可以直接super.__init__()
        super(nor_cov, self).__init__()

        '''
        #nn.Conv2d()的使用、形参与隐藏的权重参数:
            in_channels:输入张量的channels数
            out_channels:输出张量的channels数
            kernel_size:卷积核大小,可以是=3(3*3),也可以是=(3,5)(3*5)
            stride = 1:卷积核移动步长
            padding = 0:padding即填充,padding=1即上下左右都多填充一行默认0
            dilation = 1:空洞卷积,=1表示不采用,>1采用
            groups = 1:分组卷积,=1表示不采用
            bias = True:是否要添加偏置参数作为可学习参数的一个
            padding_mode = 'zeros':padding的填充模式,默认全填0
        '''
        self.cov = nn.Conv2d(in_channel, out_channel, 3, 1, 1)

        #如果设置了标准化，则使用nn.BatchNorm2d进行标准化(输入为其特征数量)
        self.normalize = normalize
        if normalize:
            self.nor = nn.BatchNorm2d(out_channel)

        #激活函数，用nn.ReLU()
        self.relu = nn.ReLU()

        #如果设置了dropout,则用nn.Dropout进行drop out(输入为其被drop out的概率p)
        self.dropout = dropout
        if dropout:
            self.drop = nn.Dropout(p=0.5)

    def forward(self, x):
        #这里单层结构就是卷积、标准化、激活函数、dropout
        x = self.cov(x)

        if self.normalize:
            x = self.nor(x)
        x = self.relu(x)

        if self.dropout:
            x = self.drop(x)

        return x
    

class dou_cov(nn.Module):
    """双层卷积网络
    很简单,即利用两次单层卷积
    """
    def __init__(self, channel, dropout=False, normalize=False):
        super(dou_cov, self).__init__()
        self.cov1 = nor_cov(in_channel=channel, out_channel=channel, dropout=dropout, normalize=normalize)
        self.cov2 = nor_cov(in_channel=channel, out_channel=channel, dropout=dropout, normalize=normalize)

    def forward(self, x):
        x = self.cov1(x)
        x = self.cov2(x)
        return x
    

class CNN_net(nn.Module):
    """自定义卷积网络
    """

    def __init__(self):
        super(CNN_net, self).__init__()

        self.cov1 = nor_cov(in_channel=3, out_channel=64, dropout=True, normalize=True)
        # self.dou_cov1 = dou_cov(channel=64, dropout=True, normalize=True)
        # self.dou_cov2 = dou_cov(channel=64, dropout=True, normalize=True)
        self.res1 = residual_block(64, dropout=False)
        self.res1111 = residual_block(64, dropout=False)
        self.res2 = residual_block(64, dropout=False)

        #Max池化函数，主要参数为kernel_size核大小，stride步长，padding补充0的层数
        self.pool1 = nn.MaxPool2d(2, 2)

        self.cov2 = nor_cov(in_channel=64, out_channel=128, dropout=True, normalize=True)
        # self.dou_cov3 = dou_cov(channel=128, dropout=True, normalize=True)
        # self.dou_cov4 = dou_cov(channel=128, dropout=True, normalize=True)
        self.res3 = residual_block(128, dropout=False)
        self.res31 = residual_block(128, dropout=False)
        self.res4 = residual_block(128, dropout=False)

        self.pool2 = nn.MaxPool2d(4, 4)

        self.cov3 = nor_cov(in_channel=128, out_channel=256, dropout=True, normalize=True)
        # self.dou_cov5 = dou_cov(channel=256, dropout=True, normalize=True)
        # self.dou_cov6 = dou_cov(channel=256, dropout=True, normalize=True)
        self.res5 = residual_block(256, dropout=False)
        self.res6 = residual_block(256, dropout=False)
        self.res51 = residual_block(256, dropout=False)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.cov4 = nor_cov(in_channel=256, out_channel=512, dropout=True, normalize=True)
        # self.dou_cov7 = dou_cov(channel=512, dropout=True, normalize=True)
        # self.dou_cov8 = dou_cov(channel=512, dropout=True, normalize=True)
        self.res7 = residual_block(512, dropout=False)
        self.res8 = residual_block(512, dropout=False)

        self.pool4 = nn.MaxPool2d(2, 2)

        self.cov5 = nor_cov(in_channel=512, out_channel=256, dropout=True, normalize=True)
        # self.dou_cov7 = dou_cov(channel=512, dropout=True, normalize=True)
        # self.dou_cov8 = dou_cov(channel=512, dropout=True, normalize=True)
        self.res9 = residual_block(256, dropout=False)
        self.res10 = residual_block(256, dropout=False)

        self.pool5 = nn.MaxPool2d(2, 2)

        self.cov6 = nor_cov(in_channel=256, out_channel=128, dropout=True, normalize=True)
        # self.dou_cov7 = dou_cov(channel=512, dropout=True, normalize=True)
        # self.dou_cov8 = dou_cov(channel=512, dropout=True, normalize=True)
        self.res11 = residual_block(128, dropout=False)
        self.res12 = residual_block(128, dropout=True)
        self.res111 = residual_block(128, dropout=True)

        self.pool6 = nn.MaxPool2d(4, 4)

        #线性映射函数nn.Linear,，参数为输入大小和输出大小
        self.fc1 = nn.Linear(128 * 1 * 1, 200)

    def forward(self, x):
        #网络结构为6层，每层为单层卷积、两层残差网络、一个池化层，最后线性输出
        #print(x.shape)
        x = self.cov1(x)
        # x = self.dou_cov1(x)
        # x = self.dou_cov2(x)
        x = self.res1(x)
        #x = self.res1111(x)
        x = self.res2(x)
        x = self.pool1(x)
        
        #print(x.shape)
        x = self.cov2(x)
        # x = self.dou_cov3(x)
        # x = self.dou_cov4(x)
        x = self.res3(x)
        #x = self.res31(x)
        x = self.res4(x)
        x = self.pool2(x)
        
        #print(x.shape)
        x = self.cov3(x)
        # x = self.dou_cov5(x)
        # x = self.dou_cov6(x)
        x = self.res5(x)
        #x = self.res51(x)
        x = self.res6(x)
        x = self.pool3(x)
        '''
        print(x.shape)
        x = self.cov4(x)
        # x = self.dou_cov7(x)
        # x = self.dou_cov8(x)
        x = self.res7(x)
        x = self.res8(x)
        x = self.pool4(x)
        
        print(x.shape)
        x = self.cov5(x)
        # x = self.dou_cov5(x)
        # x = self.dou_cov6(x)
        #x = self.res9(x)
        x = self.res10(x)
        x = self.pool5(x)
        '''
        #print(x.shape)
        x = self.cov6(x)
        # x = self.dou_cov7(x)
        # x = self.dou_cov8(x)
        x = self.res11(x)
        #x = self.res111(x)
        x = self.res12(x)
        x = self.pool6(x)

        #print(x.shape)
        x = x.reshape(x.shape[0], -1)

        #print(x.shape)
        x = self.fc1(x)

        return x

class Lab2Model(object):
    #继承object类，能具有许多高级功能

    def __init__(self, batch_size=50, num_workers=10, seed=0):
        #随机种子设置
        self.seed = seed
        self.setup_seed()

        #用自己定义的继承类Data读取训练集数据
        #这里 ToTensor 会把 numpy 类型转换为 tensor 类型，并对数据归一化到 [0, 1]
        #train_dataset = Data(type_="train", transform=transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(64)]),begin=0,end=None)

        #从训练数据中手动划分训练集和验证集
        #torch.utils.data.random_split()是数据随机分割函数，参数分别是Dataset的继承子类，分割大小比例列表（这里是4:1），generator随机数生成器
        #self.train_dataset, self.val_dataset = random_split(train_dataset, 
        #                                                    [int(len(train_dataset) * 0.8), len(train_dataset) - int(len(train_dataset) * 0.8)],
        #                                                   generator=torch.Generator().manual_seed(0))
        
        self.batch_size = batch_size
        self.num_workers = num_workers

        #将定义好的dataset融入DataLoader类中方便获取
        #参数是dataset，batch_size，shuffle表示每个epoch是否要乱序，num_workers表示是否多进程读取数据
        #drop_last表示当样本数不能被batch_size整除时，是否舍弃最后一批数据
        #self.train_dataloader = DataLoader(dataset=self.train_dataset,
        #                                   batch_size=batch_size,
         #                                  shuffle=True,
         #                                  num_workers=num_workers,
         #                                  drop_last=True)
       # self.val_dataloader = DataLoader(dataset=self.val_dataset,
         #                                batch_size=batch_size,
          #                               shuffle=False,
          #                               num_workers=num_workers,
          #                               drop_last=True)

        self.net = None
        self.lr = None
        self.optimizer = None
        self.device = None
        self.schedule = None
        self.fig_name = None
        self.loss_list = {"train": [], "val": []}

    def train(self, lr=0.01, epochs=10, device="cuda", wait=8, lrd=False, fig_name="lab2"):
        #选择设备—是否使用GPU
        self.device = torch.device(device)

        self.lr = lr
        self.fig_name = fig_name

        #定义网络net为CNN_net()
        self.net = CNN_net().to(self.device)

        #固定的优化器结构，参数1是固定的，2是学习率lr
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)

        #用来完成学习率衰减的函数，在发现loss不再降低或者acc不再提高之后，降低学习率。
        #torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode=‘min’, factor=0.1, patience=10,
        #verbose=False, threshold=0.0001, threshold_mode=‘rel’, cooldown=0, min_lr=0, eps=1e-08)
        #optimizer是优化器，mode=min或者max决定loss的目标是小还是大，factor决定学习率的缩小比例
        #patience决定几次不再降低后缩小学习率，cooldown表示是否等待几个epoch再进行检测
        if lrd:
            self.schedule = ReduceLROnPlateau(self.optimizer, 'min', patience=1, verbose=True)

        #求参数总数
        total_params = sum([param.nelement() for param in self.net.parameters() if param.requires_grad])

        print(">>> Total params: {}".format(total_params))

        print(">>> Start training")
        min_val_loss = np.inf
        min_val_loss_acc = 0.0
        delay = 0
        self.net.load_state_dict(torch.load('/kaggle/input/classification/network_params.pth'))
        '''
        for epoch in range(epochs):

            # train train data
            #用self.train_dataloader读取并循环训练每个batch的数据
            #tqdm能显示进度条
            for data in tqdm(self.train_dataloader):
                #获取数据和标签，然后训练数据并更新参数
                labels, inputs = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.net(inputs)
                loss = nn.CrossEntropyLoss()(outputs, target=labels)
                loss.backward()
                self.optimizer.step()
            
            train_loss = 0.0
            val_loss = 0.0
            val_acc = 0.0
            with torch.no_grad():
                #再次输入训练集的数据（不作参数更新），计算训练集的交叉熵损失（按batch平均）
                for data in self.train_dataloader:
                    labels, inputs = data
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.net(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, target=labels)
                    train_loss += loss.item()

                train_loss = train_loss / len(self.train_dataloader)
                self.loss_list['train'].append(train_loss)

                #输入验证集的数据（不作参数更新），计算验证集的交叉熵损失（按batch平均）和准确率
                for data in self.val_dataloader:
                    labels, inputs = data
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.net(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)

                    val_loss += loss.item()
                    val_acc += self.acc(labels=labels.cpu().numpy(), outputs=outputs.detach().cpu().numpy())
                val_loss = val_loss / len(self.val_dataloader)
                val_acc = val_acc / len(self.val_dataloader)
                self.loss_list['val'].append(val_loss)
                print(f"Epoch {epoch}: train loss {train_loss:10.6f}, "
                      f"val loss {val_loss:10.6f},"
                      f"val acc {val_acc:10.6f},")

            #根据验证集损失进行学习率衰减
            if lrd:
                self.schedule.step(val_loss)

            #早停策略：如果delay次数>wait的限制则早停
            if val_loss < min_val_loss:
                min_val_loss = val_loss
                min_val_loss_acc = val_acc
                print(f"Update min_val_loss to {min_val_loss:10.6f}")
                delay = 0
            else:
                delay = delay + 1

            if delay > wait or min_val_loss_acc>0.95:
                break
                        
                '''
        print(">>> Finished training")
        return min_val_loss_acc

    def test(self):
        #读取test的数据集并导入loader里
        test_data = Data(type_="test", transform=transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(96)]))
        test_data_loader = DataLoader(dataset=test_data,
                                      batch_size=self.batch_size,
                                      shuffle=False,
                                      num_workers=self.num_workers,
                                      drop_last=False)

        #将测试集数据输入网络计算出测试集的准确率
        test_acc = 0.0
        ii=0
        for data1 in test_data_loader:
            labels, inputs = data1
            inputs = inputs.to(self.device)
            outputs = self.net(inputs)
            pre_labels = np.argmax(outputs.detach().cpu().numpy(), axis=1)
            for j in pre_labels:
                data[ii]['chart-type']=label_transform(j)
                ii+=1

        return test_acc

    #求outputs除以真实标签的准确率，输入的labels和outputs都是numpy数组
    def acc(self, labels, outputs, type_="top1"):
        acc = 0
        if type_ == "top1":
            pre_labels = np.argmax(outputs, axis=1)
            labels = labels.reshape(len(labels))
            acc = np.sum(pre_labels == labels) / len(pre_labels)

        return acc

    #设置这四个库的seed
    def setup_seed(self):
        seed = self.seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        torch.cuda.manual_seed(seed)
        random.seed(seed)

model = Lab2Model(batch_size=64, num_workers=1, seed=0)

# 最好还是使用 GPU, CPU 太慢了。。。
model.train(lr=0.001, epochs=40, device='cuda', wait=5, lrd=True, fig_name='test')
model.test()

# Some transfunction

In [ ]:
def to_logical(img):
    N = 48
    img = np.mean(img, 2)
    rows, cols = img.shape[:2]
    rc = rows // 2; cc = cols // 2
    try:
        img_crop = img[(rc-N):(rc+N), (cc-N*2):(cc+N*2)]
    except:
        img_crop = img
    if np.max(img_crop) - np.min(img_crop) < 40:
        img_crop = img
    b = (np.max(img_crop) + np.min(img_crop) + np.mean(img_crop)) / 3
    A = img > b
    if np.sum(A) < rows * cols / 2:
        return A
    else:
        return ~A

def to_num(label : str):
    labeled = ""
    for word in label:
        if word in "-0123456789.":
            labeled += word
        elif word == "(":
            if label.find("(") == 0:
                labeled += "-"
        elif word == ",":
            if label.find(".") == -1:
                l = label.split(",")
                if len(l) == 2 and len(l[1]) != 3:
                    labeled += "."
        elif word == "b":
            labeled += "6"
        elif word == "B":
            labeled += "8"
        elif word == "K":
            labeled += "000"
    try:
        return float(labeled)
    except ValueError:
        return None
    
def to_mask(img, all):
    rows = np.shape(img)[0]
    cols = np.shape(img)[1]
    mask = Image.new('L', (cols, rows), 0)
    for a in all:
        ImageDraw.Draw(mask).polygon(a, outline=1, fill=1)
    return np.array(mask) == 0

# Find Points in vBar

In [ ]:
def find_points_in_vbar(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.8:
            img_l[r] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        box = np.zeros(r_end, np.int32)
        for r in range(3, r_end):
            box[r] = np.sum(img_l[r-1:r+2, c-5:c+5])
        m = np.max(box)
        if m >= 8:
            for r in range(3, r_end):
                if box[r] > m * 0.9:
                    points.append(r)
                    break
        else:
            points.append(r_end - 5)
    return points

# Find Points in hBar

In [ ]:
def find_points_in_hbar(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    cols = np.shape(img_l)[1]
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for c in range(c_begin, cols):
        if np.sum(img_l[:r_end, c]) > len(img_l[:r_end, c]) * 0.8:
            img_l[:, c] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        r = np.argsort([label[1][0], label[2][0], label[3][0], label[4][0]])
        r = (label[r[2] + 1][1] + label[r[3] + 1][1]) // 2
        box = np.zeros(cols, np.int32)
        for c in range(c_begin, cols - 3):
            box[c] = np.sum(img_l[r-5:r+5, c-1:c+2])
        m = np.max(box)
        if m >= 8:
            for c in reversed(range(c_begin, cols - 3)):
                if box[c] > m * 0.9:
                    points.append(c)
                    break
        else:
            points.append(c_begin + 5)
    return points

# Find Points in Dot

In [ ]:
def find_points_in_dot(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.8:
            img_l[r] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        box = np.zeros(r_end, np.int32)
        for r in range(3, r_end):
            box[r] = np.sum(img_l[r-1:r+2, c-5:c+5])
        m = np.max(box)
        if m >= 8:
            for r in range(3, r_end):
                if box[r] > m * 0.8:
                    points.append(r - 10)
                    break
        else:
            points.append(r_end - 5)
    return points

# Find Points in Line

In [ ]:
def find_points_in_line(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    cols = np.shape(img_l)[1]
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.7:
            img_l[r] = 0
        points = []
    for c in range(c_begin, cols):
        if np.sum(img_l[:r_end, c]) > len(img_l[:r_end, c]) * 0.7:
            img_l[:, c] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    count = np.zeros(np.shape(img_l))
    for r in range(3, r_end):
        for c in range(c_begin, cols - 6):
            count[r, c] = np.sum(img_l[r-3:r+3, c-3:c+3])
    index = np.zeros(cols, np.int32)
    for c in range(c_begin, cols - 6):
        if np.max(count[:, c]) > 5:
            index[c] = np.argmax(count[:, c])
        else:
            index[c] = 0
    for c in range(c_begin, cols - 6):
        if abs(index[c - 1] - index[c + 1]) < 10:
            index[c] = (index[c - 1] + index[c + 1]) / 2
        elif abs(index[c - 2] - index[c + 2]) < 10:
            index[c] = (index[c - 2] + index[c + 2]) / 2
    # plt.plot(index)
    points = []
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        if index[c] != 0: points.append(index[c])
        elif index[c-1] != 0: points.append(index[c-1])
        elif index[c+1] != 0: points.append(index[c+1])
        elif index[c-2] != 0: points.append(index[c-2])
        elif index[c+2] != 0: points.append(index[c+2])
        elif index[c-3] != 0: points.append(index[c-3])
        elif index[c+3] != 0: points.append(index[c+3])
        elif index[c-4] != 0: points.append(index[c-4])
        elif index[c+4] != 0: points.append(index[c+4])
        elif index[c-5] != 0: points.append(index[c-5])
        elif index[c+5] != 0: points.append(index[c+5])
        else: points.append(np.nan)
    for i in range(len(points)):
        if not np.isnan(points[i]):
            break
    if i == len(points) - 1 and np.isnan(points[i]):
        return points
    for j in reversed(range(len(points))):
        if not np.isnan(points[j]):
            break
    for k in range(i, j + 1):
        if np.isnan(points[k]):
            points[k] = (points[i] + points[j]) / 2
    return points

# Find Points in Scat

In [ ]:
def find_points_in_scat(img, r_begin, r_end, c_begin, mask):
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    rows, cols = np.shape(img_l)
    r = np.zeros([rows, cols])
    c = np.zeros([rows, cols])
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for i in range(r_begin, r_end):
        j = 0
        begin = 0
        now = False
        for j in range(c_begin, cols):
            if img_l[i, j] and not now:
                begin = j
                now = True
            elif not img_l[i, j] and now:
                r[i, (begin + j) // 2] = j - begin
                r[i, (begin + j) // 2 - 1] = j - begin
                r[i, (begin + j) // 2 + 1] = j - begin
                now = False
    for i in range(c_begin, cols):
        j = 0
        begin = 0
        now = False
        for j in range(r_begin, r_end):
            if img_l[j, i] and not now:
                begin = j
                now = True
            elif not img_l[j, i] and now:
                c[(begin + j) // 2, i] = j - begin
                c[(begin + j) // 2 - 1, i] = j - begin
                c[(begin + j) // 2 + 1, i] = j - begin
                now = False
    size = []
    lt = []
    for i in range(r_begin, r_end):
        for j in range(c_begin, cols):
            m = max(r[i, j], c[i, j])
            if img_l[i, j] and r[i, j] > 3 and c[i, j] > 3 and m < 20:
                size.append(min(r[i, j], c[i, j]))
                lt.append([i, j])
    l = []
    count = []
    size = np.max(size) ** 2 / 4.7
    for m in lt:
        flag = True
        for i in range(len(l)):
            s = (l[i][0] / count[i] - m[0]) ** 2 + (l[i][1] / count[i] - m[1]) ** 2
            if s < size:
                flag = False
                l[i][0] += m[0]
                l[i][1] += m[1]
                count[i] += 1
                break
        if flag:
            l.append(m)
            count.append(1)
    l = np.array(l, np.float64)
    count = np.array(count, np.float64)
    l[:, 0] = l[:, 0] / count
    l[:, 1] = l[:, 1] / count
    return l

# Points to Values

In [ ]:
def points_to_vals(points, labels, axis : int = 0):
    base = []
    val = []
    for label in labels:
        if to_num(label[0]) != None:
            base.append((label[1][axis] + label[2][axis] + label[3][axis] + label[4][axis]) / 4)
            val.append(to_num(label[0]))
    N = len(val)
    if N == 0:
        return points
    if N == 1:
        return (val[0] / base[0]) * points
    base = np.array(base)
    val = np.array(val)
    sor = np.argsort(base)
    base = base[sor]
    val = val[sor]
    flag = True
    while flag:
        flag = False
        for i in range(N):
            if i > 0 and i < N - 1:
                if val[i] > max(val[i-1], val[i+1]) or val[i] < min(val[i-1], val[i+1]):
                    N -= 1
                    val = np.delete(val, i)
                    base = np.delete(base, i)
                    flag = True
                    break
    
    N = len(val)
    w = (N * np.dot(val, base) - np.sum(val) * np.sum(base)) / (N * np.dot(base, base) - np.sum(base) ** 2)
    b = np.mean(val - w * base)
    return w * np.array(points) + b

def points_to_vals_dot(points, labels):
    # 点图只需要寻找最接近的标签的 y 值
    N = len(labels)
    base = np.zeros(N)
    val = np.zeros(N)
    for i in range(N):
        base[i] = (labels[i][1][1] + labels[i][2][1] + labels[i][3][1] + labels[i][4][1]) / 4
        val[i] = to_num(labels[i][0])
    for i in range(len(points)):
        points[i] = val[np.argmin(np.abs(base - points[i]))]
    return points


# Find Values

In [ ]:
def find_vals_in_vbar(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_vbar(img, np.min(xys), np.max(xs), xlabels, mask)
    vals = points_to_vals(np.float64(points), ylabels, 1)
    return vals

def find_vals_in_hbar(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)       
    points = find_points_in_hbar(img, np.min(xys), np.max(xs), ylabels, mask)
    vals = points_to_vals(np.float64(points), xlabels, 0)
    return vals

def find_vals_in_dot(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_dot(img, np.min(xys), np.max(xs), xlabels, mask)
    vals = points_to_vals_dot(np.float64(points), ylabels)
    return vals

def find_vals_in_line(img, xlabels, ylabels, all):
    l = len(ylabels)
    xs = np.zeros(l * 4, np.int32)
    ys = np.zeros(l, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
        ys[i] = np.min([ylabels[i][1][1], ylabels[i][2][1], ylabels[i][3][1], ylabels[i][4][1]])
    mask = to_mask(img, all)   
    points = find_points_in_line(img, np.max(ys), np.max(xs), xlabels, mask)
    vals = points_to_vals(np.float64(points), ylabels, 1)
    return vals

def find_vals_in_scat(img, xlabels, ylabels, all):
    l = len(ylabels)
    xs = np.zeros(l * 4, np.int32)
    ys = np.zeros(l * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
        ys[4 * i] = ylabels[i][1][1]
        ys[4 * i + 1] = ylabels[i][2][1]
        ys[4 * i + 2] = ylabels[i][3][1]
        ys[4 * i + 3] = ylabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_scat(img, np.min(ys), np.max(ys) + 1, np.max(xs), mask)
    vals = np.zeros(np.shape(points))
    # points 的 xy 是反的 遍历中img[i, j]的 i 代表 y
    vals[:, 0] = points_to_vals(np.float64(points[:, 1]), xlabels, 0)
    vals[:, 1] = points_to_vals(np.float64(points[:, 0]), ylabels, 1)
    sor = np.argsort(vals[:, 0])
    vals[:, 0] = vals[:, 0][sor]
    vals[:, 1] = vals[:, 1][sor]
    return vals

def find_vals_in(img, img_type : str, xlabels, ylabels, all):
    if len(xlabels) == 0 or len(ylabels) == 0:
        return []
    # labels 的每个元素为字符串 label name 与四个矩形顶点坐标 ['name', [x1, y1], [x2, y2], [x3, y3], [x4, y4]]
    if img_type == "vertical_bar":
        # 返回列表 对应按顺序每个xlabel的真实值
        return find_vals_in_vbar(img, xlabels, ylabels, all)
    elif img_type == "horizontal_bar":
        # 返回列表 对应按顺序每个ylabel的真实值
        return find_vals_in_hbar(img, xlabels, ylabels, all)
    elif img_type == "dot":
        # 返回列表 对应按顺序每个xlabel的真实值
        return find_vals_in_dot(img, xlabels, ylabels, all)
    elif img_type == "line":
        # 返回列表 对应按顺序每个xlabel的真实值 (无点时对应位置为 nan 用 np.isnan 判断)
        return find_vals_in_line(img, xlabels, ylabels, all)
    elif img_type == "scatter":
        # 返回二维列表 对应每个点的真实x值与y值
        return find_vals_in_scat(img, xlabels, ylabels, all)
    else:
        return None

# Labels to Outputs

In [ ]:
def labels_to_output(img_name : str, img, type : str, xlabels, ylabels, all):
    # labels 的每个元素为字符串 label name 与四个矩形顶点坐标 ['name', [x1, y1], [x2, y2], [x3, y3], [x4, y4]]
    vals = find_vals_in(img, type, xlabels, ylabels, all)
    N = len(vals)
    x_str = img_name + "_x,"
    y_str = img_name + "_y,"
    flag = True
    if type == "vertical_bar":
        for i in range(N):
            flag = False
            if xlabels[i][0] == "": xlabels[i][0] = "0"
            x_str += xlabels[i][0].replace(",", ".").replace(";", ".").replace('\"', "") + ";" 
            if np.isnan(vals[i]) or vals[i] == np.inf or vals[i] == -np.inf:
                y_str += "0;"
            else:   
                y_str += str(vals[i]) + ";"
    elif type == "horizontal_bar":
        for i in range(N):
            flag = False
            if np.isnan(vals[i]) or vals[i] == np.inf or vals[i] == -np.inf:
                x_str += "0;"
            else:   
                x_str += str(vals[i]) + ";"
            if ylabels[i][0] == "": ylabels[i][0] = "0"
            y_str += ylabels[i][0].replace(",", ".").replace(";", ".").replace('\"', "")  + ";"
    elif type == "dot":
        for i in range(N):
            flag = False
            if xlabels[i][0] == "": xlabels[i][0] = "0"
            x_str += xlabels[i][0].replace(",", ".").replace(";", ".").replace('\"', "")  + ";"
            if np.isnan(vals[i]) or vals[i] == np.inf or vals[i] == -np.inf:
                y_str += "0;"
            else:   
                y_str += str(vals[i]) + ";"
    elif type == "line":
        for i in range(N):
            if not np.isnan(vals[i]):
                flag = False
                if xlabels[i][0] == "": xlabels[i][0] = "0"
                x_str += xlabels[i][0].replace(",", ".").replace(";", ".").replace('\"', "")  + ";"
                y_str += str(vals[i]) + ";"
    elif type == "scatter":
        for i in range(N):
            flag = False
            if np.isnan(vals[i][0]) or vals[i][0] == np.inf or vals[i][0] == -np.inf:
                x_str += "0;"
            else:   
                x_str += str(vals[i][0]) + ";"
            if np.isnan(vals[i][1]) or vals[i][1] == np.inf or vals[i][1] == -np.inf:
                y_str += "0;"
            else:   
                y_str += str(vals[i][1]) + ";"
    else:
        return -1
    if flag:
        for i in range(len(xlabels)):
            if xlabels[i][0] == "": xlabels[i][0] = "0"
            x_str += xlabels[i][0].replace(",", ".").replace(";", ".").replace('\"', "")  + ";"
            y_str += "0;"
    x_str = x_str.rstrip(';')
    y_str = y_str.rstrip(';')
    x_str += "," + type
    y_str += "," + type
    with open("/kaggle/working/submission.csv", "a") as f:
        f.write("\n"+x_str +"\n"+ y_str)
    f.close()
    return 0

# Output

In [ ]:
import traceback
def gen_train_output():
    with open("/kaggle/working/submission.csv", "w") as f:
        f.write("id,data_series,chart_type")
    f.close()
    for datai in data:
        img_name = datai['image-name'][:-4] # img name 需要去掉 .jpg
        all = []
        for i in datai['ocr'][0]:
            i = i[0]
            all.append([i[0][0], i[0][1], i[1][0], i[1][1], i[2][0], i[2][1], i[3][0], i[3][1]])
        for i in datai['ocr45'][0]:
            i = i[0]
            all.append([i[0][0], i[0][1], i[1][0], i[1][1], i[2][0], i[2][1], i[3][0], i[3][1]])
        imgsave=img_name
        xlabels=[]
        ylabels=[]
        for i in datai['row_axis']:
            xlabels.append([i[1][0],i[0][0],i[0][1],i[0][2],i[0][3]])
        for x in xlabels:
            x.append(sorted([x[1],x[2],x[3],x[4]],key=lambda y:y[1])[0][0]+sorted([x[1],x[2],x[3],x[4]],key=lambda y:y[1])[1][0])
        xlabels=sorted(xlabels,key=lambda y:y[5])
        xlabels=[x[0:-1] for x in xlabels]
        for x in xlabels:
            x[1][0]=int(x[1][0])
            x[1][1]=int(x[1][1])
            x[2][0]=int(x[2][0])
            x[2][1]=int(x[2][1])
            x[3][0]=int(x[3][0])
            x[3][1]=int(x[3][1])
            x[4][0]=int(x[4][0])
            x[4][1]=int(x[4][1])
        for i in datai['col_axis']:
            if i[1][1]>0:
                ylabels.append([i[1][0],i[0][0],i[0][1],i[0][2],i[0][3]]) 
        for x in ylabels:
            x.append(sorted([x[1],x[2],x[3],x[4]],key=lambda y:y[0])[-1][1]+sorted([x[1],x[2],x[3],x[4]],key=lambda y:y[0])[-2][1])
        ylabels=sorted(ylabels,key=lambda y:y[5])
        ylabels=[x[0:-1] for x in ylabels]
        for x in ylabels:
            x[1][0]=int(x[1][0])
            x[1][1]=int(x[1][1])
            x[2][0]=int(x[2][0])
            x[2][1]=int(x[2][1])
            x[3][0]=int(x[3][0])
            x[3][1]=int(x[3][1])
            x[4][0]=int(x[4][0])
            x[4][1]=int(x[4][1])
        flag11=0
        try:
            labels_to_output(img_name, datai['image'], datai['chart-type'], xlabels, ylabels, all)
        except:
            exstr = traceback.format_exc()
            print(exstr)
            flag11=1
        if imgsave==None:
            while True:
                print(1)
        if flag11==1:
            x_str=imgsave+'_x,a;b,line'
            y_str=imgsave+'_y,0;1,line'
            with open("/kaggle/working/submission.csv", "a") as f:
                f.write("\n"+x_str +"\n"+ y_str)

In [ ]:
gen_train_output()

In [ ]:
!pip install /kaggle/input/jinja2/Jinja2-2.11.2-py2.py3-none-any.whl

In [ ]:
!pip install /kaggle/input/markupsafe/MarkupSafe-1.1.1-cp37-cp37m-manylinux2010_x86_64.whl